In [2]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder


import warnings
warnings.filterwarnings('ignore')

from scipy.stats import skew


In [3]:
df = pd.read_csv("datos/bikes_EDA_casual.csv", index_col=0)
df.head(5)

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,hum,windspeed,casual,casual_BOX
0,2018-01-01,winter,0,1,1,Monday,0,2,14.110847,80.5833,10.749882,331,16.397820
1,2018-01-02,winter,0,1,0,Tuesday,1,2,14.902598,69.6087,16.652113,131,11.472437
2,2018-01-03,winter,0,1,0,Wednesday,1,1,8.050924,43.7273,16.636703,120,11.076035
3,2018-01-04,winter,0,1,0,Thursday,1,1,8.200000,59.0435,10.739832,108,10.613970
4,2018-01-05,winter,0,1,0,Friday,1,1,9.305237,43.6957,12.522300,82,9.475537


In [4]:
lista_categoricas = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']

In [5]:
df_season = df.groupby('season')["casual"].mean().reset_index().sort_values(by = "casual")
df_season

,season,casual
3,winter,336.427778
0,autumn,729.112360
1,spring,1106.097826
2,summer,1202.611702


In [6]:
print(f'winter: 1')
print(f'autumn: {df_season.loc[0, "casual"]/df_season.loc[3, "casual"]}')
print(f'spring: {df_season.loc[1, "casual"]/df_season.loc[3, "casual"]}')
print(f'summer: {df_season.loc[2, "casual"]/df_season.loc[3, "casual"]}')

winter: 1
autumn: 2.167218070893557
spring: 3.2877719949081388
summer: 3.5746504348461565


In [7]:
df_yr = df.groupby('yr')["casual"].mean().reset_index().sort_values(by = "casual")
df_yr

,yr,casual
0,0,677.40274
1,1,1021.09589


In [8]:
print(f'0: 1')
print(f'1: {df_yr.loc[1, "casual"]/df_yr.loc[0, "casual"]}')

0: 1
1: 1.5073690000485334


In [9]:
df_mnth = df.groupby('mnth')["casual"].mean().reset_index().sort_values(by = "casual")
df_mnth

,mnth,casual
0,1,194.225806
1,2,266.035714
11,12,349.887097
10,11,610.050000
2,3,716.838710
9,10,963.870968
3,4,1013.366667
7,8,1161.919355
8,9,1172.050000
4,5,1214.274194


In [10]:
print(f'1: 1')
print(f'2: {df_mnth.loc[1, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'3: {df_mnth.loc[2, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'4: {df_mnth.loc[3, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'5: {df_mnth.loc[4, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'6: {df_mnth.loc[5, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'7: {df_mnth.loc[6, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'8: {df_mnth.loc[7, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'9: {df_mnth.loc[8, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'10: {df_mnth.loc[9, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'11: {df_mnth.loc[10, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'12: {df_mnth.loc[11, "casual"]/df_mnth.loc[0, "casual"]}')

1: 1
2: 1.3697238237596983
3: 3.690749045009135
4: 5.217466644521951
5: 6.25186846038864
6: 6.341931019210541
7: 6.490367048663012
8: 5.982311908320877
9: 6.034471018103305
10: 4.962630792227205
11: 3.1409317389138014
12: 1.8014449427005481


In [11]:
df_holiday = df.groupby('holiday')["casual"].mean().reset_index().sort_values(by = "casual")
df_holiday

,holiday,casual
0,0,838.69209
1,1,1189.00000


In [12]:
print(f'0: 1')
print(f'1: {df_holiday.loc[1, "casual"]/df_holiday.loc[0, "casual"]}')

0: 1
1: 1.417683573764639


In [13]:
df_weekday = df.groupby('weekday')["casual"].mean().reset_index().sort_values(by = "casual")
df_weekday

,weekday,casual
0,Friday,573.701923
4,Thursday,584.134615
6,Wednesday,629.125000
2,Saturday,682.250000
5,Tuesday,959.819048
3,Sunday,1165.211538
1,Monday,1344.676190


In [14]:
print(f'Monday: {df_weekday.loc[1, "casual"]/df_weekday.loc[0, "casual"]}')
print(f'Tuesday: {df_weekday.loc[5, "casual"]/df_weekday.loc[0, "casual"]}')
print(f'Sunday	: {df_weekday.loc[3, "casual"]/df_weekday.loc[0, "casual"]}')
print(f'Wednesday: {df_weekday.loc[6, "casual"]/df_weekday.loc[0, "casual"]}')
print(f'Saturday: {df_weekday.loc[2, "casual"]/df_weekday.loc[0, "casual"]}')
print(f'Friday: 1')
print(f'Thursday: {df_weekday.loc[4, "casual"]/df_weekday.loc[0, "casual"]}')

Monday: 2.3438586073832868
Tuesday: 1.6730274189622216
Sunday	: 2.0310399731836086
Wednesday: 1.0966060504483366
Saturday: 1.1892064024134752
Friday: 1
Thursday: 1.0181848654990362


In [15]:
df_weathersit = df.groupby('weathersit')["casual"].mean().reset_index().sort_values(by = "casual")
df_weathersit

,weathersit,casual
2,3,185.476190
1,2,689.882114
0,1,964.030238


In [16]:
print(f'3: 1')
print(f'2: {df_weathersit.loc[1, "casual"]/df_weathersit.loc[2, "casual"]}')
print(f'1: {df_weathersit.loc[0, "casual"]/df_weathersit.loc[2, "casual"]}')

3: 1
2: 3.719518457058768
1: 5.197595632657475


In [17]:
df_workingday = df.groupby('workingday')["casual"].mean().reset_index().sort_values(by = "casual")
df_workingday

,workingday,casual
1,1,803.167331
0,0,950.710526


In [18]:
print(f'1: 1')
print(f'0: {df_workingday.loc[0, "casual"]/df_workingday.loc[1, "casual"]}')

1: 1
0: 1.1837016895521375
